# Config enviroment

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Imports

In [2]:
import torch
from torch import nn

import numpy as np
import pandas as pd

from util_func import read_EPCOR

# Prepare data

In [3]:
data_root = "../Datasets/Time_Series_Datasets/EPCOR/cons_data.csv"

df = pd.read_csv(data_root)

# Models

# Train and Evaluate

# Main